# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies 
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,villa de soto,-30.8552,-64.9995,12.44,63,0,7.46,AR,1696374474
1,1,kolonia,50.9333,6.9500,14.38,77,0,4.12,DE,1696374313
2,2,isafjordur,66.0755,-23.1240,3.67,95,100,6.80,IS,1696374253
3,3,kenora,49.8167,-94.4337,20.99,77,100,8.49,CA,1696374475
4,4,puerto natales,-51.7236,-72.4875,5.48,88,34,3.01,CL,1696374476


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )


# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]



# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,port mathurin,-19.6833,63.4167,23.04,79,1,2.65,MU,1696374489
56,56,nacala,-14.5428,40.6728,22.31,72,0,4.08,MZ,1696374391
60,60,bethel,41.3712,-73.4140,22.20,82,0,0.00,US,1696374492
95,95,albany,42.6001,-73.9662,23.23,84,0,1.81,US,1696374502
96,96,dalbandin,28.8947,64.4101,24.69,14,0,1.05,PK,1696374503
98,98,le grau-du-roi,43.5388,4.1356,21.56,64,0,3.60,FR,1696374503
122,122,ataq,14.5377,46.8319,22.30,59,0,2.09,YE,1696374389
147,147,fort bragg,35.1390,-79.0060,25.51,68,0,1.54,US,1696374515
203,203,sheopur,25.6667,76.7000,23.22,44,0,1.31,IN,1696374530
212,212,al ghayzah,16.2079,52.1760,25.76,81,0,1.63,YE,1696374533


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
47,port mathurin,MU,-19.6833,63.4167,79,
56,nacala,MZ,-14.5428,40.6728,72,
60,bethel,US,41.3712,-73.4140,82,
95,albany,US,42.6001,-73.9662,84,
96,dalbandin,PK,28.8947,64.4101,14,
98,le grau-du-roi,FR,43.5388,4.1356,64,
122,ataq,YE,14.5377,46.8319,59,
147,fort bragg,US,35.1390,-79.0060,68,
203,sheopur,IN,25.6667,76.7000,44,
212,al ghayzah,YE,16.2079,52.1760,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
nacala - nearest hotel: Indico Apart Hotel 2
bethel - nearest hotel: Hampton Inn Danbury
albany - nearest hotel: No hotel found
dalbandin - nearest hotel: SF Al-dawood Restaurant
le grau-du-roi - nearest hotel: Hôtel Quai D´Azur
ataq - nearest hotel: الشارقة بلازا
fort bragg - nearest hotel: Airborne Inn Lodging
sheopur - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
hadibu - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
broome - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
dire dawa - nearest hotel: Adis Abeba
page - nearest hotel: Clarion Inn Page - Lake Powell
moses lake - nearest hotel: No hotel found
mcrae - nearest hotel: No hotel found
afif - nearest hotel: Samaya Royal Afif
ambato boeny - nearest hotel: A l'Auberge Manampisoa
newton - nearest hotel: Four Points by Sheraton Bos

,City,Country,Lat,Lng,Humidity,Hotel Name
47,port mathurin,MU,-19.6833,63.4167,79,Escale Vacances
56,nacala,MZ,-14.5428,40.6728,72,Indico Apart Hotel 2
60,bethel,US,41.3712,-73.4140,82,Hampton Inn Danbury
95,albany,US,42.6001,-73.9662,84,No hotel found
96,dalbandin,PK,28.8947,64.4101,14,SF Al-dawood Restaurant
98,le grau-du-roi,FR,43.5388,4.1356,64,Hôtel Quai D´Azur
122,ataq,YE,14.5377,46.8319,59,الشارقة بلازا
147,fort bragg,US,35.1390,-79.0060,68,Airborne Inn Lodging
203,sheopur,IN,25.6667,76.7000,44,No hotel found
212,al ghayzah,YE,16.2079,52.1760,81,فندق تاج العرب


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)